# Agentic fighters

## Exercise 3 - LangGraph for simplicity

### Graphs

![Graphs](./../images/Graphs.png)

Conceptos importantes:

- nodos
- relaciones
- estado
- persistencia

In [ ]:
import os

In [ ]:
if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0

In [ ]:
from IPython.display import Markdown
from typing import TypedDict, Annotated
import operator

from langchain_openai import ChatOpenAI
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, END

In [ ]:
base_model = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)

#### Estado

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

#### Tools

In [ ]:
if not os.environ.get("TAVILY_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

tool = TavilySearchResults(max_results=4)
print(type(tool))
print(tool.name)

#### Workflow con grafo (nodos y relaciones)

In [ ]:
class Agent:
    # General architecture of the graph
    ####################################
    def __init__(self, model, tools, system=""):
        self.system = system
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

        graph = StateGraph(AgentState)

        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)

        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")

        graph.set_entry_point("llm")

        self.graph = graph.compile()


    # Main nodes
    #############
    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)

        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if t['name'] not in self.tools:
                print("\n ....bad tool name....")
                result = "bad tool name, retry"
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")

        return {'messages': results}


    # Conditional edges
    ####################
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]

        return len(result.tool_calls) > 0

In [ ]:
prompt = """Eres un asistente de búsqueda inteligente. Utiliza el motor de búsqueda para buscar información. \
Te está permitido hacer múltiples llamadas (ya sea juntas o secuenciales). \
Solo debes buscar información cuando estés seguro de lo que necesitas. \
Está permitido buscar información antes de hacer una pregunta para clarificar.
"""

abot = Agent(base_model, [tool], system=prompt)

In [ ]:
from IPython.display import Image

Image(abot.graph.get_graph().draw_mermaid_png())

In [ ]:
messages = [HumanMessage(content="¿Que tiempo hace en Valencia?")]
result = abot.graph.invoke({"messages": messages})

In [ ]:
result

In [ ]:
result['messages'][-1].content

In [ ]:
messages = [HumanMessage(content="What is the weather in Valencia and Madrid?")]
result = abot.graph.invoke({"messages": messages})

In [ ]:
Markdown(result['messages'][-1].content)

In [ ]:
"""
query = "¿Qué equipo va último en la liga de fútbol española de este año? ¿De qué ciudad es ese equipo? ¿Cuál es la provincia que está al sur de dicha ciudad? Responde a todas las preguntas."
messages = [HumanMessage(content=query)]

abot = Agent(base_model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})
"""

In [ ]:
query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question."
messages = [HumanMessage(content=query)]

abot = Agent(base_model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

In [ ]:
print(result['messages'][-1].content)